In [ ]:
import tiktoken

# Initialize tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

# Encode a sample text
integers = tokenizer.encode("Akwirw ier")
print(integers)

# Print individual token IDs with their decoded values
for i in integers:
    print(f"{i} -> {tokenizer.decode([i])}")

# Encode individual subwords to understand tokenization process
print("Ak:", tokenizer.encode("Ak"))
print("w:", tokenizer.encode("w"))
print("ir:", tokenizer.encode("ir"))
print("w:", tokenizer.encode("w"))
print("space:", tokenizer.encode(" "))
print("ier:", tokenizer.encode("ier"))

# Decode the entire tokenized sequence
print("Decoded sequence:", tokenizer.decode([33901, 86, 343, 86, 220, 959]))

import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader

# Custom dataset for tokenized text
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window approach to split text into input-target pairs
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

# Function to create DataLoader
def create_dataloader(txt, batch_size=4, max_length=256, stride=128):
    tokenizer = tiktoken.get_encoding("gpt2")  # Use GPT-2 tokenizer
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

# Load text data
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# Initialize tokenizer and define embedding layers
tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)

vocab_size = 50257  # GPT-2 vocab size
output_dim = 256
max_len = 4
context_length = max_len

# Token and position embeddings
token_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

# Create DataLoader for batch size 4, max length 2, and stride 2
dataloader = create_dataloader(raw_text, batch_size=4, max_length=2, stride=2)

#
